# Interactive maps with Bokeh

**Credit**
[Vuokko Heikinheimo, Henrikki Tenkanen](https://automating-gis-processes.github.io/2017/lessons/L5/interactive-map-bokeh.html)  
Department of Geosciences & Geography, University of Helsinki  

Our ultimate goal today is to learn how to produce nice looking interactive maps using Geopandas and Bokeh.

In [ ]:
import os

import bokeh.io
import geopandas as gpd
import mapclassify

from bokeh.plotting import figure, save, show
from bokeh.models import ColumnDataSource, HoverTool, LinearColorMapper

bokeh.io.output_notebook()

## Simple interactive point plot

First, we learn the basic logic of plotting in Bokeh by making a simple interactive plot with few points.

First we need to initialize our plot by calling the `figure` object.

In Bokeh drawing points, lines or polygons are always done using list(s) of x and y coordinates. 

Let's create those

In [ ]:
# Create a list of x-coordinates
x_coords = [0,1,2,3,4]

# Create a list of y-coordinates
y_coords = [5,4,1,2,0]

Now we can plot those as points using a `.circle()` -object. Let’s give it a red color and size of 10.

In [ ]:
# Initialize the plot (p) and give it a title
p = None

# Plot it


# Show it


Finally, we can save our interactive plot into the disk with `save` -function that we imported in the beginning. All interactive plots are typically saved as `html` files which you can open in a web-browser.

In [ ]:
# Give output filepath
outfp = None

# Save the plot by passing the plot -object and output path



Let's go to the points.html file. That's how it should look like:

![sample plot](img/bokeh_plot.png)

## Creating interactive maps using Bokeh and Geopandas
Now we now khow how to make a really simple interactive point plot using Bokeh. What about creating such a map from a Shapefile of points? Of course we can do that, and we can use Geopandas for achieving that goal which is nice!

Creating an interactive Bokeh map from Shapefile(s) contains typically following steps:

1. **Read the Shapefile** into GeoDataFrame
2. **Calculate the x and y coordinates** of the geometries into separate columns
3. Convert the GeoDataFrame into a **Bokeh DataSource**
4. **Plot** the x and y coordinates as points, lines or polygons (which are in Bokeh words: circle, multi_line and patches)

Let’s practice these things and see how we can first create an interactive point map, then a map with lines, and finally a map with polygons where we also add those points and lines into our final map.

## Point map

Let’s first make a map out of the addresses that we geocoded in Module 3. 

Read the data using geopandas which is the first step.

In [ ]:
fp = ["..", "data", "Addresses", "addresses.shp"]
points = gpd.read_file(os.path.join(*fp))

#Let's convert the CRS to EPSG:3067


Let's see what we have

Okey, so we have the address and id columns plus the geometry column as attributes.

Now, as a second step, we need to **calculate the x and y coordinates of those points**. Unfortunately there is not a ready made function in geopandas to do that.

Thus, let’s create our own function called `getPointCoords()` which will return the x or y coordinate of a given geometry. It shall have two parameters: `geom` and `coord_type` where the first one should be a Shapely geometry object and coord_type should be either `'x'` or `'y'`.

In [ ]:
def getPointCoords(row, coord_type):
    """Calculates coordinates ('x' or 'y') of a Point geometry"""
    
    if coord_type == 'x':
        return row["geometry"].x
    elif coord_type == 'y':
        return row["geometry"].y

Let’s then use our function in a similar manner as we did before when classifying data using `.apply()` function.

In [ ]:
# Calculate x coordinates
points['x'] = None

# Calculate y coordinates
points['y'] = None

# Let's see what we have now


Now we have the x and y columns in our GeoDataFrame.

The third step, is to convert our DataFrame into a format that Bokeh can understand. Thus, we will convert our DataFrame into ColumnDataSource which is a Bokeh-specific way of storing the data.

**Bokeh `ColumnDataSource` do not understand Shapely geometry -objects. Thus, we need to remove the `geometry` -column before convert our DataFrame into a ColumnDataSouce.**

Let’s make a copy of our points GeoDataFrame where we drop the geometry column.

In [ ]:
# Make a copy and drop the geometry column
p_df = None

# See head


Now we can convert that pandas DataFrame into a ColumnDataSource.

In [ ]:
# Point DataSource
psource = None

# What is it?
psource

Okey, so now we have a ColumnDataSource object that has our data stored in a way that Bokeh wants it.

Finally, we can make a Point map of those points in a fairly similar manner as in the first example. Now instead of passing the coordinate lists, we can pass the data as a `source` for the plot with column names containing those coordinates.

In [ ]:
# Initialize our plot figure
p = None

# Add the points to the map from our 'psource' ColumnDataSource -object


# Show it


Now the last thing is to save our map as html file into our computer.

In [ ]:
# Output filepath
outfp = None

# Save the map



## Adding interactivity to the map
In Bokeh there are specific set of plot tools that you can add to the plot. Actually all the buttons that you see on the right side of the plot are exactly such tools. It is e.g. possible to interactively show information about the plot objects to the user when placing mouse over an object as you can see from the example on top of this page. The tool that shows information from the plot objects is an inspector called HoverTool that annotate or otherwise report information about the plot, based on the current cursor position.

Let’s see now how this can be done.

First, we need to initialize `HoverTool`

In [ ]:
my_hover = None

Then, we need to tell to the HoverTool that what information it should show to us. These are defined with tooltips like this:

From the above we can see that tooltip should be defined with a list of tuple(s) where the first item is the name or label for the information that will be shown, and the second item is the column-name where that information should be read in your data. The @ character in front of the column-name is important because it tells that the information should be taken from a column named as the text that comes after the character.

Lastly we need to add this new tool into our current plot.

Great! Let’s check again our map version

## Line map
Now we have made a nice point map out of a Shapefile. Let’s see how we can make an interactive map out of a Shapefile that represents metro lines in Helsinki. We follow the same steps than before, i.e. 

1) read the data,   
2) calculate x and y coordinates,   
3) convert the DataFrame into a ColumnDataSource,  
4) make the map and save it as html.

Read the data using geopandas which is the first step.

In [ ]:
# File path
metro_fp = ["..", "data", "Helsinki_metro", "metro.shp"]

# Read the data
metro = gpd.read_file(os.path.join(*metro_fp))

#Let's convert the CRS to EPSG:3067


# Let's see what we have


We have the address and id columns plus the geometry column as attributes.

Second step is where **calculate the x and y coordinates of the nodes of our lines**.

Let’s create our own function called `getLineCoords()` in a similar manner as previously but now we need to modify it a bit so that we can get coordinates out of the Shapely LineString object.




In [ ]:
def getLineCoords(row, coord_type):
    """Returns a list of coordinates ('x' or 'y') of a LineString geometry"""
    if coord_type == 'x':
        return list( row["geometry"].coords.xy[0] )
    elif coord_type == 'y':
        return list( row["geometry"].coords.xy[1] )

Let’s now apply our function in a similar manner as previously.

In [ ]:
# Calculate x coordinates of the line
metro['x'] = None

# Calculate y coordinates of the line
metro['y'] = None

# Let's see what we have now


The third step. Convert the DataFrame (without geometry column) into a ColumnDataSource which, as you remember, is a Bokeh-specific way of storing the data.

In [ ]:
# Make a copy and drop the geometry column
m_df = None

# Point DataSource
msource = None

Finally, we can make a map of the metro line and save it in a similar manner as earlier but now instead of plotting circle we need to use a `.multiline()` -object. Let’s define the `line_width` to be 3.

In [ ]:
# Initialize our plot figure
p = None

# Add the lines to the map from our 'msource' ColumnDataSource -object


# Show it


## Polygon map with Points and Lines

It is of course possible to add different layers on top of each other. Let’s visualize a map showing accessibility in Helsinki Region and place a metro line and the address points on top of that.

1st step: Import necessary modules and read the Shapefiles.

In [ ]:
# File paths
grid_fp = ["..", "data", "Travel_times", "TravelTimes_to_5975375_RailwayStation.shp"]

# Read files
grid = gpd.read_file(os.path.join(*grid_fp))

As usual, we need to make sure that the coordinate reference system is the same in every one of the layers. Let’s check the CRS of our layers.

In [ ]:
# Get the CRS of our grid, points, and metro




Let’s proceed and parse the x and y values of our grid. Let’s create own function for that as well.

In [ ]:
def getPolyCoords(row, coord_type):
    """Returns the coordinates ('x' or 'y') of edges of a Polygon exterior"""

    # Parse the exterior of the coordinate
    exterior = row["geometry"].exterior

    if coord_type == 'x':
        # Get the x coordinates of the exterior
        return list( exterior.coords.xy[0] )
    elif coord_type == 'y':
        # Get the y coordinates of the exterior
        return list( exterior.coords.xy[1] )

2nd step: Let’s now apply the functions that we have created and parse the x and y coordinates for all of our datasets.

In [ ]:
# Get the Polygon x and y coordinates
grid['x'] = None
grid['y'] = None

Now we have x and y coordinates for all of our datasets. Let’s see how our grid coordinates look like.

In [ ]:
# Show only head of x and y columns


Let’s now classify the travel times of our grid int 5 minute intervals until 200 minutes using a mapsal classifier called `User_Defined` that allows to set our own criteria for class intervals. But first we need to replace the No Data values with a large number so that they wouldn’t be seen as the “best” accessible areas.

In [ ]:
# Replace No Data values (-1) with large number (999)
grid = None

# Classify our travel times into 5 minute classes until 200 minutes
# Create a list of values where minumum value is 5, maximum value is 200 and step is 5.
breaks = None

# Initialize the classifier and apply it
classifier = None
pt_classif = None

# Rename the classified column


# Join it back to the grid layer
grid = None

In [ ]:
# Let's check what we have


3rd step: Let’s now convert our GeoDataFrames into Bokeh ColumnDataSources (without geometry columns)

In [ ]:
# Make a copy, drop the geometry column and create ColumnDataSource
g_df = None
gsource = None

4th step: For visualizing the Polygons we need to define the color palette that we are going to use. There are many different ones available but we are now going to use a palette called `RdYlBu` and use eleven color-classes for the values (defined as `RdYlBu11`). Let’s prepare our color_mapper.

In [ ]:
# Let's first do some coloring magic that converts the color palet into map numbers (it's okey not to understand)
from bokeh.palettes import RdYlBu11 as palette

# Create the color mapper
color_mapper = None

Now we are ready to visualize our polygons and add the metro line and the points on top of that. Polygons are visualized using patches objects in Bokeh.

In [ ]:
# Initialize our figure
p = None

# Plot grid



# Add metro on top of the same figure


# Add points on top (as black points)


# Show it
